In [1]:
from pyomo.environ import *  
import pandas as pd  
import time  

# Set building  
df = pd.read_excel('d:\\r.xlsx', sheet_name='data')  
K = df['DMU'].tolist()  
I = [col for col in df.columns if col.startswith('I')]  
J = [col for col in df.columns if col.startswith('O')]  
eps = 0.000001  

# Parameters building  
X = {  
    i: {  
        k: 0 for k in K  
    } for i in I  
}  
Y = {  
    j: {  
        k: 0 for k in K  
    } for j in J  
}  

# Populate input and output parameters  
for i in I:  
    for k in K:  
        X[i][k] = df.loc[df['DMU'] == k, i].values[0]  
for j in J:  
    for k in K:  
        Y[j][k] = df.loc[df['DMU'] == k, j].values[0]  

# ε-constrained DEA Model  
results = []  
for dmu in K:  
    # Start timer  
    start_time = time.time()  

    # Model Building  
    model = ConcreteModel()  

    # Decision variables Building  
    model.theta_r = Var(domain=NonNegativeReals)  
    model.lambda_k = Var(K, domain=NonNegativeReals)  
    model.s_i = Var(I, domain=NonNegativeReals)  
    model.s_j = Var(J, domain=NonNegativeReals)  

    # Objective Function setting  
    model.objective = Objective(  
        expr=model.theta_r + eps * (sum(model.s_i[i] for i in I) + sum(model.s_j[j] for j in J)),  
        sense=minimize  
    )  

    # Constraints setting  
    def input_constraint_rule(model, i):  
        return sum(model.lambda_k[k] * X[i][k] for k in K) + model.s_i[i] == model.theta_r * X[i][dmu]  
    model.input_constraint = Constraint(I, rule=input_constraint_rule)  

    def output_constraint_rule(model, j):  
        return sum(model.lambda_k[k] * Y[j][k] for k in K) - model.s_j[j] >= Y[j][dmu]  
    model.output_constraint = Constraint(J, rule=output_constraint_rule)  

    model.convexity_constraint = Constraint(expr=sum(model.lambda_k[k] for k in K) == 1)  

    # model solving  
    SolverFactory('glpk').solve(model)  

    # Collect results  
    efficiency = value(model.objective) - eps * (sum(model.s_i[i].value for i in I) + sum(model.s_j[j].value for j in J))  
    result = {  
        'DMU': dmu,  
        'Efficiency': round(efficiency, 3),  
        'Time (s)': round(time.time() - start_time, 3)  
    }  
    for k in K:  
        result[f'lambda_{k}'] = round(value(model.lambda_k[k]), 3)  
    for i in I:  
        result[f'slack_i_{i}'] = round(model.s_i[i].value, 3)  
    for j in J:  
        result[f'slack_j_{j}'] = round(model.s_j[j].value, 3)  
    result['epsilon'] = round(eps * (sum(model.s_i[i].value for i in I) + sum(model.s_j[j].value for j in J)), 3)  
    results.append(result)  

df_results = pd.DataFrame(results)  

filename = 'd:\\result-Epsilon-pyomo.xlsx'  
sheet_name = 'Results'  

try:  
    with pd.ExcelWriter(filename, mode='a', engine='openpyxl') as writer:  
        if sheet_name in writer.sheets:  
            # Append to the existing sheet  
            df_results.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=writer.sheets[sheet_name].max_row)  
        else:  
            # Create a new sheet and write the header  
            df_results.to_excel(writer, sheet_name=sheet_name, index=False, header=True)  
except:  
    # Create a new Excel file and write the header  
    df_results.to_excel(filename, sheet_name=sheet_name, index=False, header=True)

solver 'glpk'


ApplicationError: No executable found for solver 'glpk'